In [3]:
import itertools
from pyomo.environ import *
from pyomo.environ import value as pyoval

In [4]:
def vertex_enumerate(theta_nominal:list, theta_bounds:list, vertex:tuple = ('lb', 'lb', 'lb', 'lb')) -> ConcreteModel:
    
    m = ConcreteModel()
    m.i = Set(initialize=[0,1,2,3], ordered=True)

    m.thetas = Var(m.i, within=NonNegativeReals)
    m.delta = Var(within=NonNegativeReals)
    m.z = Var(within=NonNegativeReals)
    
    m.f1 = Constraint(expr = -m.z/1.5 + m.thetas[1] -350 <= 0)
    m.f2 = Constraint(expr = -0.5*m.z - 0.75*m.thetas[0] - m.thetas[1] - m.thetas[2] + 1388.5 <= 0)
    m.f3 = Constraint(expr = m.z - 1.5*m.thetas[0] - 2*m.thetas[1] - m.thetas[2] - 2*m.thetas[3] + 2830 <= 0)
    m.f4 = Constraint(expr = m.z - 1.5*m.thetas[0] - 2*m.thetas[1] - m.thetas[2] + 2044 <= 0)
    m.f5 = Constraint(expr = -m.z + 1.5*m.thetas[0] + 2*m.thetas[1] + m.thetas[2] + 3*m.thetas[3] - 3153 <= 0)
    
    if vertex[0]=='ub':
        m.theta0_UB = Constraint(expr = m.thetas[0] == theta_nominal[0] + theta_bounds[0][1]*m.delta)
    else:
        m.theta0_LB = Constraint(expr = m.thetas[0] == theta_nominal[0] - theta_bounds[0][0]*m.delta)
        
    if vertex[1]=='ub':
        m.theta1_UB = Constraint(expr = m.thetas[1] == theta_nominal[1] + theta_bounds[1][1]*m.delta)
    else:
        m.theta1_LB = Constraint(expr = m.thetas[1] == theta_nominal[1] - theta_bounds[1][0]*m.delta)
        
    if vertex[2]=='ub':
        m.theta2_UB = Constraint(expr = m.thetas[2] == theta_nominal[2] + theta_bounds[2][1]*m.delta)
    else:
        m.theta2_LB = Constraint(expr = m.thetas[2] == theta_nominal[2] - theta_bounds[2][0]*m.delta)
    
    if vertex[3]=='ub':
        m.theta3_UB = Constraint(expr = m.thetas[3] == theta_nominal[3] + theta_bounds[3][1]*m.delta)
    else:
        m.theta3_LB = Constraint(expr = m.thetas[3] == theta_nominal[3] - theta_bounds[3][0]*m.delta)
    
    m.objective = Objective(expr=m.delta, sense=maximize)
    
    solver = SolverFactory('gurobi')
    solver.solve(m)
    
    return pyoval(m.delta)

theta_nominal = [620, 388, 583, 313]
theta_bounds = [(10, 10), (10, 10), (10, 10), (10, 10)]

vertex_list = list(itertools.product(['ub','lb'], repeat=len(theta_nominal)))
delta_iter = list()

if len(theta_nominal) != len(theta_bounds):
    raise ValueError('Inconsistent input for nominal theta values and theta bounds')
else:
    for v in vertex_list:
        delta_iter.append((v, vertex_enumerate(theta_nominal=theta_nominal, theta_bounds=theta_bounds, vertex = v)))

min_idx = min(range(len(delta_iter)), key=lambda i:delta_iter[i][1])

print(f'Flexibility Index calculated through vertex enumeration: {delta_iter[min_idx]}')

Flexibility Index calculated through vertex enumeration: (('lb', 'lb', 'lb', 'lb'), 0.5600000000000023)
